# 🗄️ Notebook 3: Almacenamiento NoSQL - Amazon Reviews Analysis

**Curso:** INF3590 - Big Data  
**Universidad:** Pontificia Universidad Católica de Chile  
**Estudiante:** Oscar David Hospinal R.  
**Fecha:** Junio 2025

## 🎯 Objetivo
Implementar una solución de almacenamiento NoSQL utilizando TinyDB para los datos de reviews de Amazon preprocesados, organizando la información en colecciones lógicas y ejecutando consultas de filtrado y agregación.

## 📊 Datos de Entrada
- **Fuente:** Archivo `preprocessed_reviews.json` del Notebook 2
- **Registros:** 1,200 reviews limpias y enriquecidas
- **Estructura:** Documentos JSON con 12+ campos por registro

## 🏗️ Arquitectura NoSQL
- **Motor:** TinyDB (base documental embebida)
- **Organización:** 8 tablas (1 principal + 6 por categoría + 1 metadata)
- **Consultas:** Filtrado por rating, categoría y agregaciones
- **Distribución:** Simulación de particionado por categorías

In [1]:
# ===== CELDA 1: CONFIGURACIÓN INICIAL Y IMPORTS =====
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
from pathlib import Path
from tinydb import TinyDB, Query
from tinydb.operations import delete
import warnings
warnings.filterwarnings('ignore')

# Configuración de paths
BASE_PATH = Path(os.getcwd()).parent
DATA_PATH = BASE_PATH / "data"
SAMPLES_PATH = DATA_PATH / "samples"

# Configuración de base de datos
DB_PATH = DATA_PATH / "amazon_reviews.json"

# Configuración de categorías
CATEGORIES = {
    'Books': 'Entertainment',
    'Video_Games': 'Entertainment', 
    'Movies_and_TV': 'Entertainment',
    'Home_and_Kitchen': 'Home',
    'Tools_and_Home_Improvement': 'Home',
    'Patio_Lawn_and_Garden': 'Home'
}

print("✅ Configuración inicial completada")
print(f"📁 Base de datos: {DB_PATH}")
print(f"🗃️  Configuración para {len(CATEGORIES)} categorías")

✅ Configuración inicial completada
📁 Base de datos: D:\Proyectos con IA\May25\Proy03-Tarea 01 Amazon-Big-Data\amazon-big-data\data\amazon_reviews.json
🗃️  Configuración para 6 categorías


In [2]:
# ===== CELDA 2: CARGA DE DATOS PREPROCESADOS =====

print("📂 CARGANDO DATOS PREPROCESADOS\n")

# Cargar datos del notebook anterior
preprocessed_file = SAMPLES_PATH / "preprocessed_reviews.json"

if not preprocessed_file.exists():
    print(f"❌ Archivo no encontrado: {preprocessed_file}")
    print("💡 Asegúrate de ejecutar primero el Notebook 2 (Preprocesamiento)")
    raise FileNotFoundError(f"Archivo requerido no encontrado: {preprocessed_file}")

with open(preprocessed_file, 'r', encoding='utf-8') as f:
    all_reviews = json.load(f)

print(f"✅ Datos cargados exitosamente")
print(f"📊 Total de registros: {len(all_reviews):,}")
print(f"📁 Tamaño del archivo: {preprocessed_file.stat().st_size / 1024 / 1024:.2f} MB")

# Verificar estructura de datos
if all_reviews:
    sample_review = all_reviews[0]
    print(f"\n🔍 Estructura de registro:")
    print(f"   📝 Campos disponibles: {len(sample_review)}")
    print(f"   🏷️  Campos: {', '.join(sample_review.keys())}")
    
    # Verificar campos enriquecidos
    enriched_fields = ['category_group', 'analysis_type', 'original_category']
    missing_fields = [field for field in enriched_fields if field not in sample_review]
    
    if missing_fields:
        print(f"⚠️  Campos faltantes: {missing_fields}")
    else:
        print(f"✅ Todos los campos enriquecidos presentes")
else:
    print("❌ No hay datos para procesar")
    raise ValueError("Archivo de datos vacío")

📂 CARGANDO DATOS PREPROCESADOS

✅ Datos cargados exitosamente
📊 Total de registros: 1,200
📁 Tamaño del archivo: 1.04 MB

🔍 Estructura de registro:
   📝 Campos disponibles: 13
   🏷️  Campos: reviewerID, asin, reviewerName, helpful, reviewText, overall, summary, unixReviewTime, reviewTime, category_group, analysis_type, download_timestamp, original_category
✅ Todos los campos enriquecidos presentes


In [4]:
# =====REINICIAR KERNEL  =====

print("🗄️ INICIALIZANDO BASE DE DATOS NOSQL (ALTERNATIVA)\n")

# Crear nombre único para evitar conflictos
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
DB_PATH_NEW = DATA_PATH / f"amazon_reviews_{timestamp}.json"

print(f"📁 Creando nueva base de datos: {DB_PATH_NEW}")

# Crear nueva instancia de TinyDB
db = TinyDB(DB_PATH_NEW, indent=2, ensure_ascii=False)

print(f"✅ Base de datos inicializada: {DB_PATH_NEW}")

# Configurar tablas
print("\n🏗️ CREANDO ESTRUCTURA DE TABLAS:")

# Tabla principal para todos los reviews
reviews_table = db.table('reviews')
print(f"   📊 Tabla principal: 'reviews'")

# Tablas por categoría (simulando distribución)
category_tables = {}
for category in CATEGORIES.keys():
    table_name = category.lower().replace('_', '_').replace('&', 'and').replace(' ', '_')
    category_tables[category] = db.table(table_name)
    print(f"   📂 Tabla categoría: '{table_name}'")

# Tabla de metadata
metadata_table = db.table('metadata')
print(f"   📋 Tabla metadata: 'metadata'")

print(f"\n✅ Estructura creada: {len(category_tables) + 2} tablas")
print(f"📈 Tablas disponibles: {list(db.tables())}")

# Actualizar variable global para siguientes celdas
DB_PATH = DB_PATH_NEW

🗄️ INICIALIZANDO BASE DE DATOS NOSQL (ALTERNATIVA)

📁 Creando nueva base de datos: D:\Proyectos con IA\May25\Proy03-Tarea 01 Amazon-Big-Data\amazon-big-data\data\amazon_reviews_20250621_190630.json
✅ Base de datos inicializada: D:\Proyectos con IA\May25\Proy03-Tarea 01 Amazon-Big-Data\amazon-big-data\data\amazon_reviews_20250621_190630.json

🏗️ CREANDO ESTRUCTURA DE TABLAS:
   📊 Tabla principal: 'reviews'
   📂 Tabla categoría: 'books'
   📂 Tabla categoría: 'video_games'
   📂 Tabla categoría: 'movies_and_tv'
   📂 Tabla categoría: 'home_and_kitchen'
   📂 Tabla categoría: 'tools_and_home_improvement'
   📂 Tabla categoría: 'patio_lawn_and_garden'
   📋 Tabla metadata: 'metadata'

✅ Estructura creada: 8 tablas
📈 Tablas disponibles: []


In [5]:
# ===== CELDA 4: INSERCIÓN MASIVA DE DATOS =====
print("💾 INSERTANDO DATOS EN BASE NOSQL\n")

# Insertar en tabla principal
reviews_table.insert_multiple(all_reviews)
print(f"✅ {len(all_reviews):,} registros insertados en tabla principal")

# Insertar en tablas por categoría
for category in CATEGORIES.keys():
    category_reviews = [review for review in all_reviews 
                       if review.get('original_category') == category]
    if category_reviews:
        category_tables[category].insert_multiple(category_reviews)
        print(f"📁 {category}: {len(category_reviews)} registros")

print("\n✅ INSERCIÓN COMPLETADA")

💾 INSERTANDO DATOS EN BASE NOSQL

✅ 1,200 registros insertados en tabla principal
📁 Books: 200 registros
📁 Video_Games: 200 registros
📁 Movies_and_TV: 200 registros
📁 Home_and_Kitchen: 200 registros
📁 Tools_and_Home_Improvement: 200 registros
📁 Patio_Lawn_and_Garden: 200 registros

✅ INSERCIÓN COMPLETADA


In [6]:
# ===== CELDA 5: CONSULTAS BÁSICAS =====
from tinydb import Query
Review = Query()

# Consulta de filtrado
high_ratings = reviews_table.search(Review.overall >= 4.5)
print(f"🌟 Reviews con rating ≥4.5: {len(high_ratings):,}")

# Consulta por categoría
books_reviews = reviews_table.search(Review.original_category == 'Books')
print(f"📚 Reviews de Books: {len(books_reviews):,}")

print("✅ Consultas NoSQL ejecutadas exitosamente")

🌟 Reviews con rating ≥4.5: 796
📚 Reviews de Books: 200
✅ Consultas NoSQL ejecutadas exitosamente
